# Path Entry and Exit Automation

This notebook aims to analyze the entry and exit points in the dataset.

In [6]:
import cv2
import os
import json
import glob
import numpy as np

In [2]:
path_line = "./dataset/intersection_001/paths/path_3/path_line.png"

img = cv2.imread(path_line, cv2.IMREAD_GRAYSCALE)
type(img)

[ WARN:0@2.524] global loadsave.cpp:268 findDecoder imread_('./dataset/intersection_001/paths/path_3/path_line.png'): can't open/read file: check file path/integrity


NoneType

In [3]:
def right_edge_search(image: np.ndarray) -> list:
    m, n = image.shape
    x = n - 1  # rightmost column
    max_val = 0
    
    coords = []
    
    for i in range(m):
        if image[i][x] != 0 and image[i][x] > max_val:
            max_val = image[i][x]
            coords = [i, x]
            
    return coords

def left_edge_search(image: np.ndarray) -> list:
    m, n = image.shape
    x = 0  # leftmost column
    max_val = 0
    
    coords = []
    
    for i in range(m):
        if image[i][x] != 0 and image[i][x] > max_val:
            max_val = image[i][x]
            coords = [i, x]
            
    return coords

def top_edge_search(image: np.ndarray) -> list:
    n = image.shape[0]
    y = 0
    max_val = 0
    
    coords = []
    
    for i in range(n):
        if image[y][i] != 0 and image[y][i] > max_val:
            max_val = image[y][i]
            coords = [y, i]
            
    return coords

def bottom_edge_search(image: np.ndarray) -> list:
    m, n = image.shape
    y = m - 1  # bottom row
    max_val = 0
    
    coords = []
    
    for i in range(n):
        if image[y][i] != 0 and image[y][i] > max_val:
            max_val = image[y][i]
            coords = [y, i]
            
    return coords

In [4]:
def find_entry_coords(image: np.ndarray) -> list:
    return bottom_edge_search(image)

def find_exit_coords(image: np.ndarray) -> list:
    top = top_edge_search(image)
    if top:
        return top
    right = right_edge_search(image)
    if right:
        return right
    left = left_edge_search(image)
    if left:
        return left
    else:
        return []

In [13]:
path_line1 = "./dataset/intersection_002/paths/path_1/"
path_line2 = "./dataset/intersection_002/paths/path_2/"
path_line3 = "./dataset/intersection_001/paths/path_3/"

paths = [path_line1, path_line2, path_line3]

In [9]:
import glob
import shutil

paths = glob.glob('./dataset/*/paths/*/*.json')
print(paths) 

[]


In [10]:
for path in paths:
    dir_name = os.path.dirname(path)
    
    
    out_file = os.path.join(dir_name, "path_line_ee.json")
    
    shutil.move(path, out_file)
    print(f"Moved {path} to {out_file}")

Moved ./dataset/intersection_001/paths/path_1/fine_line_ee.json to ./dataset/intersection_001/paths/path_1/path_line_ee.json
Moved ./dataset/intersection_001/paths/path_2/fine_line_ee.json to ./dataset/intersection_001/paths/path_2/path_line_ee.json
Moved ./dataset/intersection_001/paths/path_3/fine_line_ee.json to ./dataset/intersection_001/paths/path_3/path_line_ee.json
Moved ./dataset/intersection_002/paths/path_1/fine_line_ee.json to ./dataset/intersection_002/paths/path_1/path_line_ee.json
Moved ./dataset/intersection_002/paths/path_2/fine_line_ee.json to ./dataset/intersection_002/paths/path_2/path_line_ee.json
Moved ./dataset/intersection_003/paths/path_1/fine_line_ee.json to ./dataset/intersection_003/paths/path_1/path_line_ee.json
Moved ./dataset/intersection_003/paths/path_2/fine_line_ee.json to ./dataset/intersection_003/paths/path_2/path_line_ee.json
Moved ./dataset/intersection_003/paths/path_3/fine_line_ee.json to ./dataset/intersection_003/paths/path_3/path_line_ee.json


In [7]:
paths = glob.glob('./dataset/train/*/paths/*/')
print(paths) 

r = range(113, 904+1)
r = [f"{i:03d}" for i in r]
print(r)
new_paths = [p for p in paths if any(sub in p for sub in r)]
print(new_paths)

['./dataset/train/intersection_001/paths/path_1/', './dataset/train/intersection_001/paths/path_2/', './dataset/train/intersection_001/paths/path_3/', './dataset/train/intersection_002/paths/path_1/', './dataset/train/intersection_002/paths/path_2/', './dataset/train/intersection_003/paths/path_1/', './dataset/train/intersection_003/paths/path_2/', './dataset/train/intersection_003/paths/path_3/', './dataset/train/intersection_004/paths/path_1/', './dataset/train/intersection_004/paths/path_2/', './dataset/train/intersection_004/paths/path_3/', './dataset/train/intersection_005/paths/path_1/', './dataset/train/intersection_005/paths/path_2/', './dataset/train/intersection_005/paths/path_3/', './dataset/train/intersection_006/paths/path_1/', './dataset/train/intersection_006/paths/path_2/', './dataset/train/intersection_006/paths/path_3/', './dataset/train/intersection_007/paths/path_1/', './dataset/train/intersection_007/paths/path_2/', './dataset/train/intersection_007/paths/path_3/',

In [8]:
failed = []

for path in paths:
    path = os.path.join(path, "path_line.png")
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    
    entry = find_entry_coords(img)
    exit = find_exit_coords(img)
    
    if not entry or not exit:
        print(f"Entry or exit coordinates not found for {path}")
        failed.append(path)
        continue
    
    ee = {
        "entry": {"x": entry[0], "y": entry[1]},
        "exit": {"x": exit[0], "y": exit[1]},
    }
    
    dir_name = os.path.dirname(path)
    out_file = os.path.join(dir_name, "path_line_ee.json")
    del_file = os.path.join(dir_name, "fine_line_ee.json")
    if os.path.exists(del_file):
        os.remove(del_file)
    
    with open(out_file, "w") as f:
        json.dump(ee, f, indent=4)
        
    print(f"Saved {out_file} with entry and exit coordinates: {ee}")
    
print(f"Failed to find entry or exit coordinates for {failed}" if failed else "All paths processed successfully")

Saved ./dataset/train/intersection_001/paths/path_1/path_line_ee.json with entry and exit coordinates: {'entry': {'x': 399, 'y': 204}, 'exit': {'x': 187, 'y': 0}}
Saved ./dataset/train/intersection_001/paths/path_2/path_line_ee.json with entry and exit coordinates: {'entry': {'x': 399, 'y': 229}, 'exit': {'x': 153, 'y': 399}}
Saved ./dataset/train/intersection_001/paths/path_3/path_line_ee.json with entry and exit coordinates: {'entry': {'x': 399, 'y': 217}, 'exit': {'x': 0, 'y': 223}}
Saved ./dataset/train/intersection_002/paths/path_1/path_line_ee.json with entry and exit coordinates: {'entry': {'x': 399, 'y': 197}, 'exit': {'x': 0, 'y': 269}}
Saved ./dataset/train/intersection_002/paths/path_2/path_line_ee.json with entry and exit coordinates: {'entry': {'x': 399, 'y': 198}, 'exit': {'x': 151, 'y': 0}}
Saved ./dataset/train/intersection_003/paths/path_1/path_line_ee.json with entry and exit coordinates: {'entry': {'x': 399, 'y': 161}, 'exit': {'x': 190, 'y': 0}}
Saved ./dataset/trai

In [11]:
paths_json = glob.glob('./dataset/*/paths/*/*.json')
paths_json = [os.path.dirname(path) for path in paths_json]
paths_png = glob.glob('./dataset/*/paths/*/*.png')
paths_png = [os.path.dirname(path) for path in paths_png]
paths_npy = glob.glob('./dataset/*/paths/*/*.npy')
paths_npy = [os.path.dirname(path) for path in paths_npy]

print(paths_json == paths_png == paths_npy)

True
